# Purpose: Use dimensions.ai as the source of summary statistics of research publication by category, try to identify categories with highest growth (hence more potential)
last updated on 3/20/2023
1. Collect research cateogies and codes from https://app.dimensions.ai/browse/categories/publication/for
2. Visit each catogory and crawl research data
3. Analyze growth trend and compare across time and different sectors

References: 
1. https://towardsdatascience.com/how-to-use-selenium-to-web-scrape-with-example-80f9b23a843a

In [ ]:
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \AppleWebKit/537.36 (KHTML, like Gecko) \Chrome/90.0.4430.212 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'})

In [ ]:
def get_soup(url,HEADERS):
    r = requests.get(url, headers=HEADERS).text
#     soup = BeautifulSoup(r, 'html.parser')
    soup = BeautifulSoup(r, 'lxml')
    return soup
soup= get_soup("https://app.dimensions.ai/browse/categories/publication/for",HEADERS)

In [ ]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

soup = BeautifulSoup(urlopen("https://app.dimensions.ai/browse/categories/publication/for"))
result = soup.find_all("div", {"class":"mycategories"})

for res in result:
    print(res.decode_contents().strip())

Note: used selenium instead of BeautifulSoup because the category data was loaded after loading the site

In [1]:
# use the latest selenium 4, there are syntax changes from previous version
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pickle

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:01<00:00, 6.43MB/s]


In [ ]:
driver.get("https://app.dimensions.ai/browse/categories/publication/for")

In [ ]:
all_href = driver.find_elements(By.XPATH,  "//*[contains(@href, '/discover/publication?and_facet_for')]")

In [ ]:
# Find by div class failed because can't collect the href links
# level_1 = driver.find_elements("xpath",'//div[@class="sc-2f5ewc-3 bVEbpe textcrop"]')
# level_2 = driver.find_elements("xpath",'//div[@class="sc-2f5ewc-0 gIuuzb textcrop"]')

In [ ]:
research_collection = {}
for i in range(len(all_href)):
    research_collection[all_href[i].text] = all_href[i].get_attribute('href')

In [ ]:
research_data = {}
for item in list(research_collection.items()):
    name, link = item[0],item[1]
    driver.get(link.replace("discover/publication?","analytics/publication/overview/timeline?"))
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='analytics_infinite_scroll_container']/div[2]/div[2]/div[2]/div[2]/fieldset/span[3]"))).click()
    WebDriverWait(driver, 10)
    # Reference: https://www.softwaretestingmaterial.com/how-to-handle-web-tables-in-selenium-python/
    columns_header = driver.find_elements(By.XPATH,"//table[@class= 'table table--analytical']/thead/tr/th")
#     row_header = driver.find_elements(By.XPATH,"//table[@class= 'table table--analytical']/tbody/tr/th")
    row_data = driver.find_elements(By.XPATH,"//table[@class= 'table table--analytical']/tbody/tr/td")
    data = {}
    for i in range(len(columns_header)):
        data[columns_header[i].text] = row_data[i].text
    research_data[name] = data

In [ ]:
driver.close()

In [ ]:
# Reference: https://pynative.com/python-save-dictionary-to-file/
with open('research_data.pkl', 'wb') as fp:
    pickle.dump(research_data, fp)

In [ ]:
with open('research_data.pkl', 'rb') as fp:
    research_data = pickle.load(fp)

In [ ]:
import pandas as pd
df_data = pd.DataFrame(columns = ['Code', 'Name', 2014, 2015,2016,2017,2018,2019,2020,2021,2022,2023])

In [ ]:
for key,value in research_data.items():
    value_data = { int(k):int(v.replace(",","")) for k,v in value.items() }
    code = int(key.split()[0])
    df = pd.DataFrame([value_data])
    df['Code'] = code
    df['Name'] = key
    df_data = pd.concat([df_data,df],ignore_index=True)

In [ ]:
df_data['CAGR-2022'] = ((df_data[2022]/df_data[2014])**(1/8)-1)*100
df_data['CAGR-2022'] = df_data['CAGR-2022'].astype(float).round(2)
df_data['Category-Code'] = df_data.apply(lambda x: str(x['Code'])[0:2], axis=1)
def get_rates(year_list):
    result = []
    for i in range(1, len(year_list)):
        cal = (year_list[i] - year_list[i-1])/year_list[i-1]*100
        result.append(round(cal,2))
    return result

df_data['YoY rates'] = df_data.apply(lambda x: get_rates([x[2014], x[2015],x[2016],x[2017],x[2018],x[2019],x[2020], x[2021], x[2022]]), axis=1)

In [ ]:
with pd.option_context('display.max_rows', None,'display.max_columns', None):
    display(df_data.sort_values(by='CAGR-2022', ascending=False)[['Category-Code','Code','Name',2022,'CAGR-2022','YoY rates']])

In [ ]:
df_data.to_excel("result.xlsx", index = False)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
mng = plt.get_current_fig_manager()
mng.full_screen_toggle()
df_plot = df_data[(df_data['Code'].astype(str).str.len() == 4) & (df_data['Code'].astype(str).str.startswith("46"))].set_index("Name")
df_plot = df_plot.iloc[:, 1:-1]
df_plot[df_plot.columns[df_plot.columns.isin([2014,2015,2016,2017,2018,2019,2020,2021,2022])]].T.plot()
plt.show()

# Conduct Text Analysis to Extract keywords from randomly selected abstracts in research publications

In [3]:
# pip3 install yake
import yake

In [4]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [5]:
driver.get("https://app.dimensions.ai/discover/publication?or_facet_for=80104&order=date")

In [12]:
abstract_data = driver.find_elements(By.XPATH,  "//*[@id='mainContentBlock']/div[3]/div/div[2]/div[2]/div[1]")

In [14]:
# Click show more on all publications on the first page
show_more = abstract_data[0].find_elements(By.XPATH,"//*[@data-bt='show_more']")
for more in show_more:
    more.click()

In [15]:
WebDriverWait(driver, 500)
abstract_text = abstract_data[0].find_elements(By.XPATH,"//*[@data-bt='abstract_wrapper']")

In [31]:
driver.close()

In [20]:
text_list = []
for a in abstract_text:
    text_list.append(a.text[:-4]) #remove "less" word from end

In [27]:
import pandas as pd
df_textdata = pd.DataFrame(columns = ['Abstract', 'Keywords'])

In [28]:
kw_extractor = yake.KeywordExtractor()
for text in text_list:
    keywords = kw_extractor.extract_keywords(text)
    #The lower the score, the more relevant the keyword is.
    row = pd.DataFrame([{'Abstract': text, 'Keywords': keywords}])
    df_textdata = pd.concat([df_textdata,row],ignore_index=True)

In [30]:
df_textdata.to_excel("3901 Curriculum and Pedagogy.xlsx")